In [140]:
#importación de librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

pd.set_option('display.max_columns', None) #para que muestre todas las columnas

In [108]:
#lectura de archivos csv y creación de dataframes

df_flight = pd.read_csv('Customer Flight Activity.csv', index_col='Loyalty Number')
df_loyalty = pd. read_csv('Customer Loyalty History.csv',index_col='Loyalty Number')

In [109]:
#vista inicial de los datos de df_flight

df_flight.head()

,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
Loyalty Number,,,,,,,,,
100018,2017,1,3,0,3,1521,152.0,0,0
100102,2017,1,10,4,14,2030,203.0,0,0
100140,2017,1,6,0,6,1200,120.0,0,0
100214,2017,1,0,0,0,0,0.0,0,0
100272,2017,1,0,0,0,0,0.0,0,0


In [110]:
#informacioón de los datos de df_flight

df_flight.info()

<class 'pandas.core.frame.DataFrame'>
Index: 405624 entries, 100018 to 999986
Data columns (total 9 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Year                         405624 non-null  int64  
 1   Month                        405624 non-null  int64  
 2   Flights Booked               405624 non-null  int64  
 3   Flights with Companions      405624 non-null  int64  
 4   Total Flights                405624 non-null  int64  
 5   Distance                     405624 non-null  int64  
 6   Points Accumulated           405624 non-null  float64
 7   Points Redeemed              405624 non-null  int64  
 8   Dollar Cost Points Redeemed  405624 non-null  int64  
dtypes: float64(1), int64(8)
memory usage: 30.9 MB


In [ ]:
def buscar_nulos_percent(nombre_df):
    """
    Función que busca los valores nulos y muestra el porcentaje de nulos
    """
    nulos = nombre_df.isnull().sum()
    porcentaje_nulos = (nulos/len(nombre_df))*100
    return porcentaje_nulos

In [ ]:
buscar_nulos_percent(df_flight) #no hay valores nulos

Year                           0.0
Month                          0.0
Flights Booked                 0.0
Flights with Companions        0.0
Total Flights                  0.0
Distance                       0.0
Points Accumulated             0.0
Points Redeemed                0.0
Dollar Cost Points Redeemed    0.0
dtype: float64

In [115]:
#este df tiene una cantidad alta de filas duplicadas

df_flight.duplicated().sum()

np.int64(228705)

In [116]:
#vemos que muchos Loyalty Numbers están repetidos

df_flight.index.value_counts()


Loyalty Number
678205    72
499874    48
411734    48
255475    48
528447    48
          ..
100590    24
100642    24
100644    24
100646    24
999891    24
Name: count, Length: 16737, dtype: int64

In [117]:
#Explorar la naturaleza de los datos repetidos. Se ven filas duplicadas (para drop) y filas con datos diferentes pero el mismo Loyalty Number (para mantener)

loyalty_678205   = df_flight[df_flight.index == 678205 ]
loyalty_678205 


,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
Loyalty Number,,,,,,,,,
678205,2017,1,0,0,0,0,0.0,0,0
678205,2017,1,0,0,0,0,0.0,0,0
678205,2017,1,0,0,0,0,0.0,0,0
678205,2017,2,0,0,0,0,0.0,0,0
678205,2017,2,0,0,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...
678205,2018,11,11,0,11,1177,117.0,0,0
678205,2018,11,8,1,9,1341,134.0,0,0
678205,2018,12,0,0,0,0,0.0,0,0


In [118]:
#vista inicial de los datos de df_loyalty

df_loyalty.head()

,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
Loyalty Number,,,,,,,,,,,,,,,
480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [119]:
#información de los datos de df_loyalty

df_loyalty.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16737 entries, 480934 to 652627
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country             16737 non-null  object 
 1   Province            16737 non-null  object 
 2   City                16737 non-null  object 
 3   Postal Code         16737 non-null  object 
 4   Gender              16737 non-null  object 
 5   Education           16737 non-null  object 
 6   Salary              12499 non-null  float64
 7   Marital Status      16737 non-null  object 
 8   Loyalty Card        16737 non-null  object 
 9   CLV                 16737 non-null  float64
 10  Enrollment Type     16737 non-null  object 
 11  Enrollment Year     16737 non-null  int64  
 12  Enrollment Month    16737 non-null  int64  
 13  Cancellation Year   2067 non-null   float64
 14  Cancellation Month  2067 non-null   float64
dtypes: float64(4), int64(2), object(9)
memory usage: 2.0

In [120]:
buscar_nulos_percent(df_loyalty)

Country                0.000000
Province               0.000000
City                   0.000000
Postal Code            0.000000
Gender                 0.000000
Education              0.000000
Salary                25.321145
Marital Status         0.000000
Loyalty Card           0.000000
CLV                    0.000000
Enrollment Type        0.000000
Enrollment Year        0.000000
Enrollment Month       0.000000
Cancellation Year     87.650117
Cancellation Month    87.650117
dtype: float64

In [121]:
#este df no tiene filas dupllicadas

df_loyalty.duplicated().sum()

np.int64(0)

In [122]:
df = df_flight.merge(df_loyalty,on = "Loyalty Number", how="left") #left join to keep all the rows of df_flight
df.sample(5)

,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
Loyalty Number,,,,,,,,,,,,,,,,,,,,,,,,
146702,2018,8,8,8,16,1792,179.0,577,47,Canada,British Columbia,Whistler,V6T 1Y8,Male,Bachelor,71633.0,Married,Aurora,7385.36,2018 Promotion,2018,2,NaN,NaN
468192,2017,3,0,0,0,0,0.0,0,0,Canada,Quebec,Quebec City,G1B 3L5,Male,Bachelor,73602.0,Married,Nova,6689.02,Standard,2016,5,NaN,NaN
954424,2018,3,0,0,0,0,0.0,0,0,Canada,Ontario,Kingston,M9K 2P4,Female,College,NaN,Single,Star,4698.76,Standard,2016,1,2018.0,8.0
407377,2018,12,10,0,10,2360,236.0,0,0,Canada,Ontario,Toronto,P5S 6R4,Male,Bachelor,86981.0,Married,Star,9388.16,Standard,2015,7,NaN,NaN
686471,2017,4,12,0,12,1644,164.0,0,0,Canada,Ontario,Ottawa,K1G 4Z0,Male,Master,117771.0,Single,Star,2248.45,Standard,2014,10,NaN,NaN


## Exploración General del DataFrame despues de unir las tablas

In [123]:
df.shape

(405624, 24)

In [124]:
#evaluación de valores nulos

buscar_nulos_percent(df)

Year                            0.000000
Month                           0.000000
Flights Booked                  0.000000
Flights with Companions         0.000000
Total Flights                   0.000000
Distance                        0.000000
Points Accumulated              0.000000
Points Redeemed                 0.000000
Dollar Cost Points Redeemed     0.000000
Country                         0.000000
Province                        0.000000
City                            0.000000
Postal Code                     0.000000
Gender                          0.000000
Education                       0.000000
Salary                         25.312112
Marital Status                  0.000000
Loyalty Card                    0.000000
CLV                             0.000000
Enrollment Type                 0.000000
Enrollment Year                 0.000000
Enrollment Month                0.000000
Cancellation Year              87.657535
Cancellation Month             87.657535
dtype: float64

In [125]:
df.index.value_counts() #hay Loyalty Numbers repetidos

Loyalty Number
678205    72
499874    48
411734    48
255475    48
528447    48
          ..
100590    24
100642    24
100644    24
100646    24
999891    24
Name: count, Length: 16737, dtype: int64

In [126]:
loyalty_678205  = df[df.index == 678205 ] #para visualizar como se ven los datos de un Loyalty Number. Hay meses repetidos pero no son todos filas enteras repetidas
loyalty_678205 

,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
Loyalty Number,,,,,,,,,,,,,,,,,,,,,,,,
678205,2017,1,0,0,0,0,0.0,0,0,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,67358.0,Married,Nova,20230.97,Standard,2015,9,NaN,NaN
678205,2017,1,0,0,0,0,0.0,0,0,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,67358.0,Married,Nova,20230.97,Standard,2015,9,NaN,NaN
678205,2017,1,0,0,0,0,0.0,0,0,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,67358.0,Married,Nova,20230.97,Standard,2015,9,NaN,NaN
678205,2017,2,0,0,0,0,0.0,0,0,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,67358.0,Married,Nova,20230.97,Standard,2015,9,NaN,NaN
678205,2017,2,0,0,0,0,0.0,0,0,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,67358.0,Married,Nova,20230.97,Standard,2015,9,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678205,2018,11,11,0,11,1177,117.0,0,0,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,67358.0,Married,Nova,20230.97,Standard,2015,9,NaN,NaN
678205,2018,11,8,1,9,1341,134.0,0,0,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,67358.0,Married,Nova,20230.97,Standard,2015,9,NaN,NaN
678205,2018,12,0,0,0,0,0.0,0,0,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,67358.0,Married,Nova,20230.97,Standard,2015,9,NaN,NaN


In [127]:
df.describe().T #resumen stats de columnas numéricas    

,count,mean,std,min,25%,50%,75%,max
Year,405624.0,2017.500000,0.500001,2017.00,2017.00,2017.50,2018.00,2018.00
Month,405624.0,6.500000,3.452057,1.00,3.75,6.50,9.25,12.00
Flights Booked,405624.0,4.115052,5.225518,0.00,0.00,1.00,8.00,21.00
Flights with Companions,405624.0,1.031805,2.076869,0.00,0.00,0.00,1.00,11.00
Total Flights,405624.0,5.146858,6.521227,0.00,0.00,1.00,10.00,32.00
Distance,405624.0,1208.880059,1433.155320,0.00,0.00,488.00,2336.00,6293.00
Points Accumulated,405624.0,123.692721,146.599831,0.00,0.00,50.00,239.00,676.50
Points Redeemed,405624.0,30.696872,125.486049,0.00,0.00,0.00,0.00,876.00
Dollar Cost Points Redeemed,405624.0,2.484503,10.150038,0.00,0.00,0.00,0.00,71.00
Salary,302952.0,79268.825953,34992.133508,-58486.00,59262.00,73479.00,88612.00,407228.00


In [128]:
df.describe(include='object').T #resumen stats de columnas categóricas

,count,unique,top,freq
Country,405624,1,Canada,405624
Province,405624,11,Ontario,130896
City,405624,29,Toronto,81096
Postal Code,405624,55,V6E 3D9,21984
Gender,405624,2,Female,203640
Education,405624,5,Bachelor,253752
Marital Status,405624,3,Married,235800
Loyalty Card,405624,3,Star,184224
Enrollment Type,405624,2,Standard,382200


In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 405624 entries, 100018 to 999986
Data columns (total 24 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Year                         405624 non-null  int64  
 1   Month                        405624 non-null  int64  
 2   Flights Booked               405624 non-null  int64  
 3   Flights with Companions      405624 non-null  int64  
 4   Total Flights                405624 non-null  int64  
 5   Distance                     405624 non-null  int64  
 6   Points Accumulated           405624 non-null  float64
 7   Points Redeemed              405624 non-null  int64  
 8   Dollar Cost Points Redeemed  405624 non-null  int64  
 9   Country                      405624 non-null  object 
 10  Province                     405624 non-null  object 
 11  City                         405624 non-null  object 
 12  Postal Code                  405624 non-null  object 
 13 

## Limpieza
- Eliminar columnas Cancellation Month y Cancellation year (debido al porcentaje de nulos)
- tratar los nulos en la columna (salario)
- Evaluar salarios en negativo
- eliminar duplicados


In [131]:
df.duplicated().sum()

np.int64(1864)

In [132]:
df.drop_duplicates(inplace=True)
df.duplicated().sum() #comprobamos que se eliminaron las filas duplicadas

np.int64(0)

In [133]:
drop_columns= ['Cancellation Year', 'Cancellation Month'] #eliminar columnas con 87% de nulos

df.drop(drop_columns, axis=1, inplace=True)

In [ ]:
df_salaries = np.sort(df['Salary'].unique()) #para ver los valores únicos de la columna Salary, para identificar valores negativos

df_salaries

array([-58486., -57297., -49830., ..., 397919., 407228.,     nan],
      shape=(5891,))

In [139]:
df['Salary'] = df['Salary'].abs()

df_salaries = np.sort(df['Salary'].unique()) #comprobar que se han cambiado los valores negativos

df_salaries

array([  9081.,  10605.,  12497., ..., 397919., 407228.,     nan],
      shape=(5891,))

In [141]:
#para tratar los nulos en salary: hay un 25% de nulos. Opto por tratar con Iterative Imputer

imputer = IterativeImputer(max_iter=100, random_state=42)
df['Salary'] = imputer.fit_transform(df[['Salary']])

In [142]:
buscar_nulos_percent(df) #ya no hay nulos en Salary

Year                           0.0
Month                          0.0
Flights Booked                 0.0
Flights with Companions        0.0
Total Flights                  0.0
Distance                       0.0
Points Accumulated             0.0
Points Redeemed                0.0
Dollar Cost Points Redeemed    0.0
Country                        0.0
Province                       0.0
City                           0.0
Postal Code                    0.0
Gender                         0.0
Education                      0.0
Salary                         0.0
Marital Status                 0.0
Loyalty Card                   0.0
CLV                            0.0
Enrollment Type                0.0
Enrollment Year                0.0
Enrollment Month               0.0
dtype: float64